In [66]:
import numpy as np
import pandas as pd
import unicodedata
import re
import nltk

from numpy.random import choice

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import prepare
# nltk.download('wordnet') - needed to run to download 'wordnet' resource to use lemmatize function

import warnings
warnings.filterwarnings('ignore')

# MODELING
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_json('data.json')
df

,repo,language,readme_contents
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w..."
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...
3,beetbox/beets,Python,.. image:: https://img.shields.io/pypi/v/beets...
4,scottschiller/SoundManager2,JavaScript,# SoundManager 2: JavaScript Sound for the Web...
...,...,...,...
197,andyxialm/CutoLoadingView,Java,#CutoLoadingView\n[![](https://jitpack.io/v/an...
198,rjsvieira/morphos,Java,# Morphos\n\n![Current Version](https://img.sh...
199,tuesda/SubmitDemo,Java,This is library project with a custom view tha...
200,avenwu/support,Java,Support\n========\n\nCustom Android support li...


In [3]:
df = prepare.prep_data(df, 'readme_contents')

In [4]:
df

,repo,language,readme_contents,stemmed,lemmatized,clean,stopwords_removed,doc_length,words
0,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w...",movi applic for thi project we will be build a...,movie application for this project we will be ...,movie application building single page movie a...,381,403,"[movie, application, building, single, page, m..."
1,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel there are sever guid for kernel d...,linux kernel there are several guide for kerne...,linux kernel several guide kernel developer us...,43,67,"[linux, kernel, several, guide, kernel, develo..."
2,beetbox/beets,Python,.. image:: https://img.shields.io/pypi/v/beets...,imag http img shield io pypi v beet svg target...,image http img shield io pypi v beet svg targe...,image http img shield io pypi v beet svg targe...,239,484,"[image, http, img, shield, io, pypibeet, svg, ..."
3,scottschiller/SoundManager2,JavaScript,# SoundManager 2: JavaScript Sound for the Web...,soundmanag javascript sound for the web by wra...,soundmanager javascript sound for the web by w...,soundmanager javascript sound web wrapping ext...,335,584,"[soundmanager, javascript, sound, web, wrappin..."
4,CreateJS/SoundJS,JavaScript,# SoundJS\r\n\r\nSoundJS is a library to make ...,soundj soundj is a librari to make work with a...,soundjs soundjs is a library to make working w...,soundjs soundjs make working audio web easier ...,194,316,"[soundjs, soundjs, make, working, audio, web, ..."
...,...,...,...,...,...,...,...,...,...
180,andyxialm/CutoLoadingView,Java,#CutoLoadingView\n[![](https://jitpack.io/v/an...,cutoloadingview http jitpack io v andyxialm cu...,cutoloadingview http jitpack io v andyxialm cu...,cutoloadingview http jitpack io v andyxialm cu...,72,134,"[cutoloadingview, http, jitpack, ioandyxialm, ..."
181,rjsvieira/morphos,Java,# Morphos\n\n![Current Version](https://img.sh...,morpho current version http img shield io badg...,morphos current version http img shield io bad...,morphos current version http img shield io bad...,202,669,"[morphos, current, version, http, img, shield,..."
182,tuesda/SubmitDemo,Java,This is library project with a custom view tha...,thi is librari project with a custom view that...,this is library project with a custom view tha...,custom view implement concept submit button ht...,16,93,"[custom, view, implement, concept, submit, but..."
183,avenwu/support,Java,Support\n========\n\nCustom Android support li...,support custom android support librari includ ...,support custom android support library include...,support custom android support include useful ...,94,262,"[support, custom, android, support, include, u..."


In [5]:
df.language.value_counts()

HTML                30
JavaScript          28
Python              28
Ruby                28
C++                 28
Java                24
Jupyter Notebook     6
CSS                  4
TypeScript           3
Scala                2
C#                   2
C                    2
Name: language, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   repo               185 non-null    object
 1   language           185 non-null    object
 2   readme_contents    185 non-null    object
 3   stemmed            185 non-null    object
 4   lemmatized         185 non-null    object
 5   clean              185 non-null    object
 6   stopwords_removed  185 non-null    int64 
 7   doc_length         185 non-null    int64 
 8   words              185 non-null    object
dtypes: int64(2), object(7)
memory usage: 13.1+ KB


In [7]:
from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(df, stratify=df.language, test_size=.15, random_state=123)
train, validate = train_test_split(train_validate, stratify = train_validate.language, test_size = .20, random_state = 123)

In [8]:
train.shape

(125, 9)

In [9]:
train.language.value_counts()

HTML                20
Python              19
Ruby                19
JavaScript          19
C++                 19
Java                16
Jupyter Notebook     4
C                    2
C#                   2
TypeScript           2
CSS                  2
Scala                1
Name: language, dtype: int64

In [10]:
validate.shape

(32, 9)

In [11]:
validate.language.value_counts()

Python              5
Ruby                5
JavaScript          5
C++                 5
HTML                5
Java                4
Scala               1
Jupyter Notebook    1
CSS                 1
Name: language, dtype: int64

In [12]:
# create series objects for each top_code_clean that is a string of words joined on spaces to make it 1 continious string  
javascript_words = ' '.join(train[train.language=='JavaScript'].clean)
python_words = ' '.join(train[train.language=='Python'].clean)
ruby_words = ' '.join(train[train.language=='Ruby'].clean)
html_words = ' '.join(train[train.language=='HTML'].clean)
c_plus_plus_words = ' '.join(train[train.language=='C++'].clean)
all_words = ' '.join(train.clean)

# Exploration

Explore the data that you have scraped. Here are some ideas for exploration:

- What are the most common words in READMEs?
- What does the distribution of IDFs look like for the most common words?
- Does the length of the README vary by programming language?
- Do different programming languages use a different number of unique words?

### What are the most common words in READMEs?

In [13]:
all_words_df = pd.DataFrame(all_words.split())

In [14]:
all_words_df.value_counts().head(10)

http          2454
learning       545
data           506
machine        462
python         345
html           307
code           268
build          252
tensorflow     248
io             239
dtype: int64

### What does the distribution of IDFs look like for the most common words?

In [15]:
# List of the top ten most common words across the combined readmes of the train dataset
most_common_words = [word[0] for word in list(all_words_df.value_counts().head(10).index)]
most_common_words

['http',
 'learning',
 'data',
 'machine',
 'python',
 'html',
 'code',
 'build',
 'tensorflow',
 'io']

In [16]:
def idf(most_common_words):
    # This dictionary will store how many documents each word appears in 
    appearances_dict = dict.fromkeys(most_common_words)
    
    # The total number of documents is based on the number of rows in the train dataframe
    number_of_documents = train.shape[0]
    
    # This is essentially a list containing the contents of the words column in the dataframe. It is a list of lists.
    list_of_wordlists = list(train.words.values)
    
    # Start iterating through the list of common words. We want to collect information for each one.
    for word in most_common_words:
        
        # Set the initial number of documents that the word appears in to zero
        number_of_appearances = 0
        
        # Start iterating through the list made from the words column in the dataframe 
        for words in list_of_wordlists:
            
            # If the current word is in a document, add 1 to the number of appearances and then move to the next document
            if word in words:
                number_of_appearances += 1
        
        # Once all the documents have been iterated through, add the sum total of all appearances to our appearances dictionary
        appearances_dict[word] = number_of_appearances
        
    # Create a new dictionary that will contain the IDF values for each word
    idf_dict = dict.fromkeys(most_common_words)
    
    # Start iterating through the list of common words again, using the number of appearances and the total number of documents to calculate the IDF and update the relevant key:value in the dictionary
    for word in most_common_words:
        idf_dict[word] = np.log(number_of_documents / appearances_dict[word])
        
    # Return the dictionary showing the actual number of appearances and the dictionary showing the calculated IDFs
    return appearances_dict, idf_dict

In [17]:
idf(most_common_words)

({'http': 107,
  'learning': 15,
  'data': 38,
  'machine': 20,
  'python': 28,
  'html': 52,
  'code': 66,
  'build': 49,
  'tensorflow': 5,
  'io': 53},
 {'http': 0.15548490284039496,
  'learning': 2.120263536200091,
  'data': 1.1907275775759152,
  'machine': 1.8325814637483102,
  'python': 1.4961092271270973,
  'html': 0.8770700187208738,
  'code': 0.6386589952758757,
  'build': 0.9364934391916745,
  'tensorflow': 3.2188758248682006,
  'io': 0.8580218237501793})

### Does the length of the README vary by programming language?

In [18]:
train.groupby('language').doc_length.mean().sort_values(ascending=False)

language
TypeScript          2160.000000
Python              1602.631579
Ruby                 448.263158
C#                   402.500000
C++                  396.631579
JavaScript           347.526316
Scala                292.000000
Java                 288.000000
HTML                 267.250000
C                    221.000000
CSS                  168.500000
Jupyter Notebook     111.250000
Name: doc_length, dtype: float64

In [19]:
from math import sqrt
from scipy import stats

In [20]:
# This cell block looks at every pair of languages and compares the document lengths using a t-test

# Start by developing two identical lists of all of the languages represented in the train dataset
# We will use the series above to form our list so our output is arranged in a similar order
# When this is converted to a function, we can generate the list in a more generalized manner
train_language_list_1 = list(train.groupby('language').doc_length.mean().sort_values(ascending=False).index)
train_language_list_2 = list(train.groupby('language').doc_length.mean().sort_values(ascending=False).index)

# This empty list will hold information about which pairs have been tested. 
# If the python and javascript pair has already be tested, then we do not need to test the javascript and python pair
testing_pairs = []

for language_1 in train_language_list_1: # Iterates through list 1
    for language_2 in train_language_list_2: # Iterates through list 2 in entirety for each element in list 1
        
        if language_1 == language_2: # Cannot run a t-test against itself, so skip the test if the two list elements are identical
            continue
            
        else:
            alpha = 0.05 # Set alpha
            
            # Run the t-test and store the t-statistic and the p-value
            stat, p = stats.ttest_ind(train[train.language == language_1].doc_length, train[train.language == language_2].doc_length)
            
            # If the p-value is statistically significant we print the results, otherwise we do nothing
            if p/2 < alpha:
                
                # Creating strings to represent the pair that is being tested (eg. 'Python and JavaScript' & 'JavaScript and Python')
                testing_pair_1 = language_1 + " " + language_2
                testing_pair_2 = language_2 + " " + language_1
                
                # If this unique pair has not yet been tested:
                if (testing_pair_1 not in testing_pairs) and (testing_pair_2 not in testing_pairs):
                    
                    # Add this pair to the testing_pairs list so that we do not output duplicate t-test results
                    testing_pairs.append(testing_pair_1)
                    testing_pairs.append(testing_pair_2)
                    
                    # Print the results of the test
                    print("----------------")
                    print(f"Document Length T-Test: {language_1} & {language_2}")
                    print("----------------")
                    print("Hypotheses:")
                    print(f"H_0: There is no difference in the mean document lengths of {language_1} and {language_2}")
                    print(f"H_a: There is a difference in the mean document lengths of {language_1} and {language_2}")
                    print('\n')
                    print(f"p-value: {p/2}")
                    print(f"t-statistic: {stat}")
                    print(f"We reject the null hypothesis")
                    print("\n")
                    if stat < 0:
                        print(f"The mean readme document length for {language_1} is smaller than {language_2}")
                    elif stat > 0:
                        print(f"The mean readme document length for {language_1} is larger than {language_2}")
                    print('\n','\n')
                    
                    # If the pair had already been tested, do not print any results and continue through the loop
                else:
                    continue

----------------
Document Length T-Test: TypeScript & Ruby
----------------
Hypotheses:
H_0: There is no difference in the mean document lengths of TypeScript and Ruby
H_a: There is a difference in the mean document lengths of TypeScript and Ruby


p-value: 0.0020690582683747082
t-statistic: 3.2580543145300953
We reject the null hypothesis


The mean readme document length for TypeScript is larger than Ruby

 

----------------
Document Length T-Test: TypeScript & C++
----------------
Hypotheses:
H_0: There is no difference in the mean document lengths of TypeScript and C++
H_a: There is a difference in the mean document lengths of TypeScript and C++


p-value: 0.00221159171653436
t-statistic: 3.228402181104798
We reject the null hypothesis


The mean readme document length for TypeScript is larger than C++

 

----------------
Document Length T-Test: TypeScript & JavaScript
----------------
Hypotheses:
H_0: There is no difference in the mean document lengths of TypeScript and JavaScri

### Do different programming languages use a different number of unique words?

In [21]:
unique_word_count = {}

for language in train_language_list_1:
    unique_word_count[language] = len(set(' '.join(train[train.language==language].clean).split()))

In [22]:
unique_word_count = pd.DataFrame.from_dict(unique_word_count, orient='index').rename(columns={0: 'num_unique_words'})

In [23]:
unique_word_count.sort_values(by='num_unique_words', ascending=False)

,num_unique_words
Python,6154
Ruby,2263
C++,2116
JavaScript,1980
HTML,1662
TypeScript,1294
Java,1283
C#,344
Jupyter Notebook,270
C,226


While this gives us a total count of the number of unique words each programming language has connected to it, this is likely to be correlated both to the total document length AND the number of observations that each language has. Languages with more observations likely have more opportunities to expand the unique word list. 

A better metric would be the average number of unique words that each language has. To do this we will need to add a column to our dataframe.

In [24]:
train['unique_word_count'] = train.words.apply(lambda x: len(set(x)))

In [25]:
train

,repo,language,readme_contents,stemmed,lemmatized,clean,stopwords_removed,doc_length,words,unique_word_count
26,censusreporter/censusreporter,HTML,About Census Reporter\n=====================\n...,about censu report the unit state censu bureau...,about census reporter the united state census ...,census reporter united state census bureau pro...,1532,2789,"[census, reporter, united, state, census, bure...",864
135,StarterSquad/ngseed,HTML,ngseed\n======\n\nAngularJS/RequireJS seed pro...,ngseed angularj requirej seed project document...,ngseed angularjs requirejs seed project docume...,ngseed angularjs requirejs seed documentation ...,107,249,"[ngseed, angularjs, requirejs, seed, documenta...",144
77,igrigorik/decisiontree,Ruby,# Decision Tree\n\nA Ruby library which implem...,decis tree a rubi librari which implement id3 ...,decision tree a ruby library which implement i...,decision tree ruby implement id3 information g...,97,262,"[decision, tree, ruby, implement, id3, informa...",129
84,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div align center img src http www tensorflow o...,div align center img src http www tensorflow o...,div align center img src http tensorflow image...,377,1698,"[div, align, center, img, src, http, tensorflo...",328
1,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel there are sever guid for kernel d...,linux kernel there are several guide for kerne...,linux kernel several guide kernel developer us...,43,67,"[linux, kernel, several, guide, kernel, develo...",49
...,...,...,...,...,...,...,...,...,...,...
16,designmodo/Flat-UI,JavaScript,\n# [Flat UI Free](https://designmodo.github.i...,flat ui free http designmodo github io flat ui...,flat ui free http designmodo github io flat ui...,flat ui free http designmodo io flat ui flat u...,189,416,"[flat, ui, free, http, designmodo, io, flat, u...",181
5,musescore/MuseScore,C++,![MuseScore](mscore/data/musescore_logo_full.p...,musescor mscore data musescor logo full png mu...,musescore mscore data musescore logo full png ...,musescore mscore data musescore logo full png ...,236,616,"[musescore, mscore, data, musescore, logo, ful...",314
48,Hextris/hextris,JavaScript,"Hextris\n==========\n\n<img src=""images/twitte...",hextri img src imag twitter opengraph png widt...,hextris img src image twitter opengraph png wi...,hextris img src image twitter opengraph png wi...,112,149,"[hextris, img, src, image, twitter, opengraph,...",101
147,patriciogonzalezvivo/KinectCoreVision,C++,"<iframe class=""roundPhoto"" src=""http://player....",ifram class roundphoto src http player vimeo c...,iframe class roundphoto src http player vimeo ...,iframe class roundphoto src player vimeo video...,53,123,"[iframe, class, roundphoto, src, player, vimeo...",87


Now we can run a similar set of t-tests to compare the unique word count for each programming language

In [26]:
train.groupby('language').unique_word_count.mean().sort_values(ascending=False)

language
TypeScript          700.000000
Python              489.526316
Ruby                207.368421
C#                  188.000000
JavaScript          180.210526
C++                 170.947368
Scala               154.000000
Java                131.062500
C                   121.500000
HTML                119.050000
CSS                 100.500000
Jupyter Notebook     71.500000
Name: unique_word_count, dtype: float64

In [27]:
# This cell block looks at every pair of languages and compares the document lengths using a t-test

# Start by developing two identical lists of all of the languages represented in the train dataset
# We will use the languages represented in the array above so that the output below is sorted in a similar manner
# When this is converted to a function, we can generate the list in a more generalized manner
train_language_list_1 = list(train.groupby('language').unique_word_count.mean().sort_values(ascending=False).index)
train_language_list_2 = list(train.groupby('language').unique_word_count.mean().sort_values(ascending=False).index)

# This empty list will hold information about which pairs have been tested. 
# If the python and javascript pair has already be tested, then we do not need to test the javascript and python pair
testing_pairs = []

for language_1 in train_language_list_1: # Iterates through list 1
    for language_2 in train_language_list_2: # Iterates through list 2 in entirety for each element in list 1
        
        if language_1 == language_2: # Cannot run a t-test against itself, so skip the test if the two list elements are identical
            continue
            
        else:
            alpha = 0.05 # Set alpha
            
            # Run the t-test and store the t-statistic and the p-value
            stat, p = stats.ttest_ind(train[train.language == language_1].unique_word_count, train[train.language == language_2].unique_word_count)
            
            # If the p-value is statistically significant we print the results, otherwise we do nothing
            if p/2 < alpha:
                
                # Creating strings to represent the pair that is being tested (eg. 'Python and JavaScript' & 'JavaScript and Python')
                testing_pair_1 = language_1 + " " + language_2
                testing_pair_2 = language_2 + " " + language_1
                
                # If this unique pair has not yet been tested:
                if (testing_pair_1 not in testing_pairs) and (testing_pair_2 not in testing_pairs):
                    
                    # Add this pair to the testing_pairs list so that we do not output duplicate t-test results
                    testing_pairs.append(testing_pair_1)
                    testing_pairs.append(testing_pair_2)
                    
                    # Print the results of the test
                    print("----------------")
                    print(f"Unique Word Count T-Test: {language_1} & {language_2}")
                    print("----------------")
                    print("Hypotheses:")
                    print(f"H_0: There is no difference in the mean number of unique words for {language_1} and {language_2}")
                    print(f"H_a: There is a difference in the mean number of unique words for {language_1} and {language_2}")
                    print('\n')
                    print(f"p-value: {p/2}")
                    print(f"t-statistic: {stat}")
                    print(f"We reject the null hypothesis")
                    print("\n")
                    if stat < 0:
                        print(f"The mean number of unique words for {language_1} is smaller than {language_2}")
                    elif stat > 0:
                        print(f"The mean number of unique words for {language_1} is larger than {language_2}")
                    print('\n','\n')
                    
                    # If the pair had already been tested, do not print any results and continue through the loop
                else:
                    continue

----------------
Unique Word Count T-Test: TypeScript & Ruby
----------------
Hypotheses:
H_0: There is no difference in the mean number of unique words for TypeScript and Ruby
H_a: There is a difference in the mean number of unique words for TypeScript and Ruby


p-value: 0.0056065203235507784
t-statistic: 2.8085396735768975
We reject the null hypothesis


The mean number of unique words for TypeScript is larger than Ruby

 

----------------
Unique Word Count T-Test: TypeScript & JavaScript
----------------
Hypotheses:
H_0: There is no difference in the mean number of unique words for TypeScript and JavaScript
H_a: There is a difference in the mean number of unique words for TypeScript and JavaScript


p-value: 0.0023768248278700226
t-statistic: 3.196281182487515
We reject the null hypothesis


The mean number of unique words for TypeScript is larger than JavaScript

 

----------------
Unique Word Count T-Test: TypeScript & C++
----------------
Hypotheses:
H_0: There is no differenc

In [28]:
# train['silly_readme'] = train.words.apply(lambda x: '00s' in x)
# train[train.silly_readme == True]

# This originally was added to the dataframe to find the readme that had '00s' repeated in it thousands of times
# Turns out that this readme included massive text tables, and the numeric values were being parsed by the prep_data function as words

In [29]:
# train.drop(columns=['silly_readme'], inplace=True)

# Model

In [30]:
tfidf = TfidfVectorizer()
tfidf

TfidfVectorizer()

In [31]:
# X_train, X_validate, X_test are transformed to sparse array representing TF-IDF values for every word

X_train = tfidf.fit_transform(train.clean)

X_validate = tfidf.transform(validate.clean)

X_test = tfidf.transform(test.clean)

In [32]:
# y_train, y_validate, y_test are simply the actual language labels

y_train = train.language
y_validate = validate.language
y_test = test.language

## Baseline

We will create two baseline models. 

The first will predict a language at random using weighted probabilities based on the distribution of the languages in the train dataset. 

The second will be based on the most common language in the train dataset.

### Random Baseline Model

In [33]:
def baseline_language_model(train, num_observations):
    '''
    This function takes in a train dataset and identifies the proportions for the languages represented in that set.
    It then creates an array num_observations long where each element is one of the languages randomly selected (weighted by the languages proportion)
    The array is returned to be used as a series of predictions to compare other models against.
    '''
    possible_languages = list(train.language.value_counts().index)
    probabilities = []
    for language in possible_languages:
        probabilities.append(train.language.value_counts()[language] / train.language.shape[0])
    baseline_predictions = choice(possible_languages, num_observations, p=probabilities)
    return baseline_predictions

In [34]:
baseline_train_pred_random = baseline_language_model(train, num_observations = train.language.shape[0])
baseline_train_pred_random[:10]

array(['C++', 'C++', 'HTML', 'Ruby', 'C++', 'C++', 'Python', 'C++',
       'JavaScript', 'C++'], dtype='<U16')

In [35]:
train_results = pd.DataFrame(dict(actual=y_train))
train_results['base_random'] = baseline_train_pred_random
train_results

,actual,base_random
26,HTML,C++
135,HTML,C++
77,Ruby,HTML
84,C++,Ruby
1,C,C++
...,...,...
16,JavaScript,C#
5,C++,HTML
48,JavaScript,HTML
147,C++,C++


In [37]:
print('Baseline Model: Weighted Random Selection')
print('Accuracy: {:.2%}'.format(accuracy_score(train_results.actual, train_results.base_random)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train_results.base_random, train_results.actual))
print('---')
print(classification_report(train_results.actual, train_results.base_random))

Baseline Model: Weighted Random Selection
Accuracy: 16.80%
---
Confusion Matrix
actual            C  C#  C++  CSS  HTML  Java  JavaScript  Jupyter Notebook  \
base_random                                                                   
C                 0   0    1    0     0     0           0                 0   
C#                0   0    0    0     1     0           1                 0   
C++               1   0    2    1     4     6           2                 1   
CSS               0   0    0    0     0     0           0                 1   
HTML              0   0    2    0     5     3           3                 0   
Java              1   1    2    0     2     1           1                 0   
JavaScript        0   0    5    0     3     3           4                 1   
Jupyter Notebook  0   0    0    0     0     0           3                 0   
Python            0   0    2    0     3     2           2                 0   
Ruby              0   1    5    1     1     0      

### Baseline Model: Mode Selection

In [38]:
train_results['base_mode'] = train.language.value_counts().idxmax()
train_results

,actual,base_random,base_mode
26,HTML,C++,HTML
135,HTML,C++,HTML
77,Ruby,HTML,HTML
84,C++,Ruby,HTML
1,C,C++,HTML
...,...,...,...
16,JavaScript,C#,HTML
5,C++,HTML,HTML
48,JavaScript,HTML,HTML
147,C++,C++,HTML


In [40]:
print('Baseline Model: Mode Selection')
print('Accuracy: {:.2%}'.format(accuracy_score(train_results.actual, train_results.base_mode)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train_results.base_mode, train_results.actual))
print('---')
print(classification_report(train_results.actual, train_results.base_mode))

Baseline Model: Mode Selection
Accuracy: 16.00%
---
Confusion Matrix
actual     C  C#  C++  CSS  HTML  Java  JavaScript  Jupyter Notebook  Python  \
base_mode                                                                      
HTML       2   2   19    2    20    16          19                 4      19   

actual     Ruby  Scala  TypeScript  
base_mode                           
HTML         19      1           2  
---
                  precision    recall  f1-score   support

               C       0.00      0.00      0.00         2
              C#       0.00      0.00      0.00         2
             C++       0.00      0.00      0.00        19
             CSS       0.00      0.00      0.00         2
            HTML       0.16      1.00      0.28        20
            Java       0.00      0.00      0.00        16
      JavaScript       0.00      0.00      0.00        19
Jupyter Notebook       0.00      0.00      0.00         4
          Python       0.00      0.00      0.00     

## Logistic Regression

In [41]:
lm = LogisticRegression().fit(X_train, y_train)

In [42]:
train_results['lm_predicted'] = lm.predict(X_train)
train_results

,actual,base_random,base_mode,lm_predicted
26,HTML,C++,HTML,HTML
135,HTML,C++,HTML,HTML
77,Ruby,HTML,HTML,Ruby
84,C++,Ruby,HTML,C++
1,C,C++,HTML,Python
...,...,...,...,...
16,JavaScript,C#,HTML,JavaScript
5,C++,HTML,HTML,C++
48,JavaScript,HTML,HTML,JavaScript
147,C++,C++,HTML,C++


In [43]:
print('Accuracy: {:.2%}'.format(accuracy_score(train_results.actual, train_results.lm_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train_results.lm_predicted, train_results.actual))
print('---')
print(classification_report(train_results.actual, train_results.lm_predicted))

Accuracy: 89.60%
---
Confusion Matrix
actual        C  C#  C++  CSS  HTML  Java  JavaScript  Jupyter Notebook  \
lm_predicted                                                              
C++           1   0   19    0     0     0           0                 0   
HTML          0   2    0    1    20     0           0                 1   
Java          0   0    0    0     0    16           0                 0   
JavaScript    0   0    0    1     0     0          19                 0   
Python        1   0    0    0     0     0           0                 2   
Ruby          0   0    0    0     0     0           0                 1   

actual        Python  Ruby  Scala  TypeScript  
lm_predicted                                   
C++                0     0      0           0  
HTML               0     0      1           0  
Java               0     0      0           0  
JavaScript         0     0      0           2  
Python            19     0      0           0  
Ruby               0    1

## Decision Tree

In [67]:
clf = DecisionTreeClassifier(max_depth = 15, random_state = 123).fit(X_train, y_train)

In [68]:
clf_pred = clf.predict(X_train)
clf_pred[:10]

array(['HTML', 'HTML', 'Ruby', 'C++', 'C', 'HTML', 'C++', 'Java',
       'TypeScript', 'Ruby'], dtype=object)

In [69]:
train_results['clf_predicted'] = clf_pred
train_results

,actual,base_random,base_mode,lm_predicted,clf_predicted
26,HTML,C++,HTML,HTML,HTML
135,HTML,C++,HTML,HTML,HTML
77,Ruby,HTML,HTML,Ruby,Ruby
84,C++,Ruby,HTML,C++,C++
1,C,C++,HTML,Python,C
...,...,...,...,...,...
16,JavaScript,C#,HTML,JavaScript,JavaScript
5,C++,HTML,HTML,C++,C++
48,JavaScript,HTML,HTML,JavaScript,JavaScript
147,C++,C++,HTML,C++,C++


In [70]:
print('Decision Tree Model')
print('Accuracy: {:.2%}'.format(accuracy_score(train_results.actual, train_results.clf_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train_results.clf_predicted, train_results.actual))
print('---')
print(classification_report(train_results.actual, train_results.clf_predicted))

Decision Tree Model
Accuracy: 94.40%
---
Confusion Matrix
actual            C  C#  C++  CSS  HTML  Java  JavaScript  Jupyter Notebook  \
clf_predicted                                                                 
C                 2   0    0    0     0     0           0                 0   
C#                0   2    0    1     0     0           0                 0   
C++               0   0   17    0     0     0           0                 0   
CSS               0   0    0    1     0     0           0                 1   
HTML              0   0    2    0    20     0           0                 2   
Java              0   0    0    0     0    16           0                 0   
JavaScript        0   0    0    0     0     0          19                 0   
Jupyter Notebook  0   0    0    0     0     0           0                 1   
Python            0   0    0    0     0     0           0                 0   
Ruby              0   0    0    0     0     0           0                

## Random Forest

In [71]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=3, 
                            random_state=123).fit(X_train, y_train)

In [72]:
rf_pred = rf.predict(X_train)
rf_pred[:10]

array(['HTML', 'HTML', 'Ruby', 'C++', 'HTML', 'HTML', 'C++', 'Java',
       'Python', 'Ruby'], dtype=object)

In [73]:
train_results['rf_predicted'] = rf_pred

In [74]:
print('Random Forest Model')
print('Accuracy: {:.2%}'.format(accuracy_score(train_results.actual, train_results.rf_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train_results.rf_predicted, train_results.actual))
print('---')
print(classification_report(train_results.actual, train_results.rf_predicted))

Random Forest Model
Accuracy: 80.80%
---
Confusion Matrix
actual        C  C#  C++  CSS  HTML  Java  JavaScript  Jupyter Notebook  \
rf_predicted                                                              
C++           0   0   13    0     0     0           0                 0   
HTML          1   1    6    2    20     2           2                 4   
Java          0   0    0    0     0    14           0                 0   
JavaScript    0   1    0    0     0     0          17                 0   
Python        0   0    0    0     0     0           0                 0   
Ruby          1   0    0    0     0     0           0                 0   

actual        Python  Ruby  Scala  TypeScript  
rf_predicted                                   
C++                0     0      0           0  
HTML               0     1      0           1  
Java               0     0      1           0  
JavaScript         0     0      0           0  
Python            19     0      0           1  
Ruby 